## Import

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

## Read Data

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [3]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [4]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


## Modelling

In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Save scaler for future work

In [8]:
import pickle
pickle.dump(sc, open('sc.pickle', 'wb'))

In [9]:
Xtrain = torch.from_numpy(X_train).float()
Xtest = torch.from_numpy(X_test).float()

In [10]:
ytrain = torch.from_numpy(y_train)
ytest = torch.from_numpy(y_test)

In [11]:
Xtrain.shape, ytrain.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [12]:
Xtest.shape, ytest.shape

(torch.Size([311, 2]), torch.Size([311]))

In [13]:
input_size=2
output_size=2
hidden_size=10

In [14]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [15]:
model = Net()

In [16]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [17]:
epochs = 100

In [18]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Y_pred = model(Xtrain)
  loss = loss_fn(Y_pred, ytrain)
  loss.backward()
  optimizer.step()
  print('Epoch', epoch, 'loss', loss.item())

Epoch 0 loss 0.6258062720298767
Epoch 1 loss 0.5864478349685669
Epoch 2 loss 0.5502930879592896
Epoch 3 loss 0.5202417969703674
Epoch 4 loss 0.4928760230541229
Epoch 5 loss 0.4663565754890442
Epoch 6 loss 0.4400068521499634
Epoch 7 loss 0.4135989248752594
Epoch 8 loss 0.3869844675064087
Epoch 9 loss 0.3605712354183197
Epoch 10 loss 0.33454740047454834
Epoch 11 loss 0.3093789219856262
Epoch 12 loss 0.2856661379337311
Epoch 13 loss 0.2640449106693268
Epoch 14 loss 0.24472397565841675
Epoch 15 loss 0.22778232395648956
Epoch 16 loss 0.21317827701568604
Epoch 17 loss 0.20080089569091797
Epoch 18 loss 0.19026145339012146
Epoch 19 loss 0.18116918206214905
Epoch 20 loss 0.17334693670272827
Epoch 21 loss 0.16658364236354828
Epoch 22 loss 0.16074134409427643
Epoch 23 loss 0.15588413178920746
Epoch 24 loss 0.1518249809741974
Epoch 25 loss 0.14849860966205597
Epoch 26 loss 0.14577896893024445
Epoch 27 loss 0.1434808075428009
Epoch 28 loss 0.1413809359073639
Epoch 29 loss 0.1394231915473938
Epoch 3

In [19]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.6965, -0.5971],
         [ 0.4648,  0.4236],
         [-0.5152, -0.4653],
         [-0.3549,  1.0781],
         [ 0.7935, -0.2453],
         [-0.6750, -0.4961],
         [-0.4204,  0.7933],
         [-0.0262, -0.2128],
         [ 1.1619,  0.4457],
         [-0.2803, -0.8764]], requires_grad=True), Parameter containing:
 tensor([ 0.1352,  0.5762,  0.1342,  0.5722, -0.0127,  0.6255,  0.3634,  0.6535,
          0.5605,  0.1210], requires_grad=True), Parameter containing:
 tensor([[ 0.5279,  0.1811,  0.4672, -0.0531, -0.6590,  0.2994, -0.3117,  0.1240,
          -0.1266,  0.2877],
         [-0.3570,  0.4101, -0.2478,  0.5456,  0.2854, -0.2392,  0.3276,  0.3720,
           0.5992,  0.1509],
         [-0.0258,  0.5162, -0.2151,  0.2517,  0.7093, -0.0744,  0.4304, -0.0263,
           0.6077, -0.2077],
         [ 0.5136, -0.0617,  0.5159, -0.3166, -0.1846,  0.0510, -0.4339,  0.2066,
          -0.0948,  0.3731],
         [ 0.3455, -0.1829,  0.1766, -0.2761, -

Check model 

In [20]:
new_data = torch.from_numpy(sc.transform(np.array([[42, 50000]]))).float()

In [21]:
y_cust = model(new_data)
y_cust

tensor([[-0.9621, -0.4814]], grad_fn=<LogSoftmaxBackward>)

In [22]:
_, predicted = torch.max(y_cust, -1)
predicted

tensor([1])

Save model for future work

In [23]:
torch.save(model, 'customer_buy.pt')

In [25]:
torch.save(model.state_dict(), 'customer_buy_state_dict')

In [26]:
# !zip -r customer_buy_state_dict.zip customer_buy_state_dict

In [27]:
# new_predictor = Net()

In [28]:
# from google.colab import files


In [29]:
# files.download('customer_buy_state_dict.zip')